In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import dataframe_manip as hp
import pytz

%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
data = hp.csv_to_timeseries_df(filepath='../wifi_data_until_20190204.csv', timezone=pytz.timezone('US/Pacific'))

In [6]:
temp = data
temp = pd.to_datetime(temp.index, utc=True)

TypeError: argument of type 'Timestamp' is not iterable

In [4]:
print(temp)

DatetimeIndex(['2018-07-18 23:10:14+00:00', '2018-07-18 23:15:23+00:00',
               '2018-07-18 23:20:38+00:00', '2018-07-18 23:25:06+00:00',
               '2018-07-18 23:30:06+00:00', '2018-07-18 23:35:08+00:00',
               '2018-07-18 23:40:12+00:00', '2018-07-18 23:45:16+00:00',
               '2018-07-18 23:50:19+00:00', '2018-07-18 23:55:12+00:00',
               ...
               '2019-02-04 19:00:08+00:00', '2019-02-04 19:05:08+00:00',
               '2019-02-04 19:10:07+00:00', '2019-02-04 19:15:07+00:00',
               '2019-02-04 19:20:08+00:00', '2019-02-04 19:25:08+00:00',
               '2019-02-04 19:30:08+00:00', '2019-02-04 19:35:12+00:00',
               '2019-02-04 19:40:08+00:00', '2019-02-04 19:45:08+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=57772, freq=None)
